In [68]:
from numpy import *

def loadDataSet():
    return mat([
        [-4.06724228, 0.21993975],
        [0.73633558, -1.41299247],
        [-2.59754537, 3.15378974],
        [4.49190084, 3.46005807],
        [-3.62111442, -2.36505947],
        [2.21588922, -2.88365904],
        [-2.38799628, 2.96837672],
        [2.6265299, 3.10868015],
        [-3.53973889, -2.89384326],
        [2.65077367, -2.79019029],
        [-2.46154315, 2.78737555],
        [2.6265299, 3.10868015],
    ])
         
def distEclud(vecA, vecB): # 计算欧氏距离
    return sqrt(sum(power(vecA - vecB, 2)))
    
def randCent(dataSet, k): # 从样本集的数据范围内，随机生成k个随机值
    n = shape(dataSet)[1]
    centroids = mat(zeros((k, n)))
    
    for j in range(n):
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = minJ + rangeJ * random.rand(k, 1)
    
    return centroids

def kMeans(dataSet, k, distMeas=distEclud, createCent=randCent):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m, 2))) # 用于存储迭代样本归属的簇
    centroids = createCent(dataSet, k) # 随机选择k个质心
    clusterChanged = True
    
    while clusterChanged: # 当没有样本的聚类再次发生变化，停止
        clusterChanged = False
        for i in range(m):
            minDist = inf; minIndex = -1;
            for j in range(k):
                distJI = distMeas(centroids[j, :], dataSet[i, :])
                if distJI < minDist:
                    minDist = distJI
                    minIndex = j
            if clusterAssment[i, 0] != minIndex: # 聚类结果发生变化 
                clusterChanged = True
            clusterAssment[i, :] = minIndex, minDist ** 2 # 更新聚类结果
        for cent in range(k):
            ptsInClust = dataSet[nonzero(clusterAssment[:, 0].A == cent)[0]]
            centroids[cent, :] = mean(ptsInClust, axis = 0) # 更新质心
        print 'centroids=', centroids
        
    return centroids, clusterAssment

def biKmeans(dataSet, k, distMeas=distEclud):
    m = shape(dataSet)[0] # 样本数m
    clusterAssment = mat(zeros((m, 2))) # 初始化分簇信息 1 存质心index 2 存 到质心的方差
    centroid0 = mean(dataSet, axis=0).tolist()[0] # 第一个质心
    centList = [centroid0]
    
    for j in range(m):# 初始化m样本到全局质心的方差
        clusterAssment[j, 1] = distMeas(mat(centroid0), dataSet[j, :]) ** 2
        
    while (len(centList) < k):
        lowestSSE = inf
        for i in range(len(centList)): # 遍历所有的质心，尝试二分
            ptsIncurrCluster = dataSet[nonzero(clusterAssment[:, 0].A == i)[0], :] # 抽取该簇下的样本
            centroidMat, splitClustAss = kMeans(ptsIncurrCluster, 2, distMeas) # 2分聚类
            sseSplit = sum(splitClustAss[:, 1]) # 统计分类后的总误差
            sseNotSplit = sum(clusterAssment[nonzero(clusterAssment[:, 0].A != i)[0], 1]) # 统计其他簇的总误差
            
            print 'sseSplit, notSplit:', sseSplit, sseNotSplit
            
            if (sseSplit + sseNotSplit) < lowestSSE: # 判断二分后 误差是否变小
                bestCentToSplit = i # 二分前的簇编号
                bestNewCents = centroidMat # 二分后的质心
                bestClustAss = splitClustAss.copy() # 二分后的样本分簇结果
                lowestSSE = sseSplit + sseNotSplit # 最小方差
                
        bestClustAss[nonzero(bestClustAss[:, 0].A == 1)[0], 0] = len(centList) # 二分出的1号簇，编号为L号簇
        bestClustAss[nonzero(bestClustAss[:, 0].A == 0)[0], 0] = bestCentToSplit # 二分出的0号簇，保留为原编号
        
        print 'the bestCentToSplit is:', bestCentToSplit
        print 'the len of bestClustAss is: ', len(bestClustAss)
        
        centList[bestCentToSplit] = bestNewCents[0, :] # 原簇质心的更新
        centList.append(bestNewCents[1,:]) # 追加新生成的簇
        clusterAssment[nonzero(clusterAssment[:, 0].A == bestCentToSplit)[0], :] = bestClustAss # 更新样本的簇判定结果
    
    return centList, clusterAssment
                

In [69]:
print '======== kMeans'
centroids, clustAssing = kMeans(loadDataSet(), 4)
print 'final centroids', centroids
print 'clustAssing', clustAssing

print '========= biKmeans'
centList, clustAssing = biKmeans(loadDataSet(), 4)
print 'final centList', centList
print 'clustAssing', clustAssing

======== kMeans
centroids= [[ 0.38297931  3.09782673]
 [        nan         nan]
 [-3.74269853 -1.67965433]
 [ 1.86766616 -2.3622806 ]]
centroids= [[ 0.38297931  3.09782673]
 [        nan         nan]
 [-3.74269853 -1.67965433]
 [ 1.86766616 -2.3622806 ]]
final centroids [[ 0.38297931  3.09782673]
 [        nan         nan]
 [-3.74269853 -1.67965433]
 [ 1.86766616 -2.3622806 ]]
clustAssing [[ 2.          3.7137863 ]
 [ 3.          2.18105683]
 [ 0.          8.88665921]
 [ 0.         17.01444771]
 [ 2.          0.48456291]
 [ 3.          0.39309478]
 [ 0.          7.69506301]
 [ 0.          5.03363706]
 [ 2.          1.51544738]
 [ 3.          0.79636408]
 [ 0.          8.18768794]
 [ 0.          5.03363706]]
========= biKmeans
centroids= [[-0.99224664  1.50533304]
 [ 1.86766616 -2.3622806 ]]
centroids= [[-0.99224664  1.50533304]
 [ 1.86766616 -2.3622806 ]]
sseSplit, notSplit: 140.62652998340909 0.0
the bestCentToSplit is: 0
the len of bestClustAss is:  12
centroids= [[ 3.24832021  3.22